# MVE Axiom Prediction

% https://colab.research.google.com/drive/1DIQm9rOx2mT1bZETEeVUThxcrP1RKqAn#scrollTo=IS1dPinuyPCy reference

In [1]:
%env CUDA_VISIBLE_DEVICES=""

env: CUDA_VISIBLE_DEVICES=""


# TODO




* Try to get a reasonable score?
* TODO add batchnorm [done]
* TODO add residual connections [done]
* TODO add other directions? [done]
* TODO add FileWriter [done]
* TODO separate itno functions [done]

# Imports

In [2]:
import torch
from torch.nn import Embedding
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, Linear
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.transforms import to_undirected, ToUndirected

/home/eholden/.pyenv/versions/3.7.5/lib/python3.7/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
import sys, os
from pathlib import Path

sys.path.append(str(Path(os.path.abspath("")).parent))

import config
from dataset import get_data_loader, BenchmarkType
from stats_writer import Writer

## CONSTANTS

In [4]:
TRAIN_ID = '../id_files/train.txt'
#TRAIN_ID = "../id_files/validation.txt"

VAL_ID = "../id_files/validation.txt"
BENCHMARK_TYPE = BenchmarkType("deepmath")

In [5]:
torch.manual_seed(1234567)

## Load dataset

In [6]:
# dataset_params = {'transform': ToUndirected()}
dataset_params = {"transform": None}

# transform = None

In [7]:
data_instance = get_data_loader(TRAIN_ID, BENCHMARK_TYPE, **dataset_params)
#val_data = get_data_loader(VAL_ID, BENCHMARK_TYPE, **dataset_params)

Dataset: TorchMemoryDataset(22179)


## Data point check

In [8]:
next(iter(data_instance))

DataBatch(x=[46632], edge_index=[2, 77783], premise_index=[968], conjecture_index=[64], name=[64], y=[968], batch=[46632], ptr=[65])

In [9]:
data = next(iter(data_instance))[0]
print(data)
print(data.keys)
print(data.num_nodes)
print(data.num_edges)
print(data.num_node_features)
print(data.has_isolated_nodes())
print(data.is_directed())

Data(x=[982], edge_index=[2, 1771], premise_index=[14], conjecture_index=[1], name='t29_bvfunc11', y=[14])
['conjecture_index', 'x', 'edge_index', 'premise_index', 'y', 'name']
982
1771
1
True
True


## Model

In [10]:
def get_dense_output_network(no_dense_layers, hidden_dim, task, dropout_rate):
    if task == "premise":
        return DensePremiseOutput(no_dense_layers, hidden_dim, dropout_rate)

In [11]:
class DensePremiseOutput(torch.nn.Module):
    def __init__(self, no_dense_layers, hidden_dim, dropout_rate):
        super(DensePremiseOutput, self).__init__()

        self.no_dense_layers = no_dense_layers
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate

        self.lin = nn.ModuleList()
        for _ in range(no_dense_layers):
            self.lin.append(nn.Linear(hidden_dim, hidden_dim))

        # Add output layer
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, x, premise_index):

        # Extract premises
        x = x[premise_index]

        # Dense feedforward
        for i in range(self.no_dense_layers):
            x = self.lin[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

        # Output layer
        x = self.out(x)
        x = x.squeeze(-1)

        return x

In [12]:
GCN_NORMALISATION = {"batch": nn.BatchNorm1d, "layer": nn.LayerNorm}

In [13]:
def build_normalisation_layers(normaliser, hidden_dim, num_normalisation_layers):

    lns = nn.ModuleList()
    for _ in range(num_normalisation_layers):
        lns.append(normaliser(hidden_dim))
    return lns


def build_conv_model(num_convolutional_layers, hidden_dim, flow):

    convs = nn.ModuleList()
    for _ in range(num_convolutional_layers):
        convs.append(get_conv_layer(hidden_dim, hidden_dim, flow))

    return convs


def get_conv_layer(input_dim, hidden_dim, flow):
    return pyg_nn.GCNConv(input_dim, hidden_dim, flow=flow)


def build_merge_linear_layers(num_linear_layers, hidden_dim):

    lin = nn.ModuleList()
    for _ in range(num_linear_layers):
        lin.append(Linear(hidden_dim * 2, hidden_dim))

    return lin

In [14]:
class GCNDirectional(torch.nn.Module):
    def __init__(self, hidden_dim, num_convolutional_layers, dropout_rate, normalisation, skip_connection):
        super(GCNDirectional, self).__init__()

        self.flow = "target_to_source"  # Sets direction to bottom up

        # Set variables
        self.hidden_dim = hidden_dim
        self.num_convolutional_layers = num_convolutional_layers
        self.dropout_rate = dropout_rate
        self.skip_connection = skip_connection

        # Add convolutional layers
        self.convs = build_conv_model(num_convolutional_layers, hidden_dim, self.flow)

        # Add normalisation layers used in between graph convolutions
        if normalisation is None:
            self.lns = None
        else:
            self.normaliser = GCN_NORMALISATION[normalisation]
            self.lns = build_normalisation_layers(self.normaliser, hidden_dim, num_convolutional_layers - 1)

    def forward(self, x, edge_index):

        # Iterate over each convolutional sequence
        for i in range(self.num_convolutional_layers):

            conv_out = self.convs[i](x, edge_index)
            # Check if applying skip connection
            if self.skip_connection:
                x = x + conv_out
            else:
                x = conv_out

            emb = x
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

            if self.lns is not None and not i == self.num_convolutional_layers - 1:  # Apply normalisation
                x = self.lns[i](x)

        return emb, x


sub = GCNDirectional(
    hidden_dim=32, num_convolutional_layers=3, dropout_rate=0.25, normalisation="batch", skip_connection=False
)
sub

GCNDirectional(
  (convs): ModuleList(
    (0): GCNConv(32, 32)
    (1): GCNConv(32, 32)
    (2): GCNConv(32, 32)
  )
  (lns): ModuleList(
    (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [15]:
class GCNBiDirectional(torch.nn.Module):
    def __init__(self, hidden_dim, num_convolutional_layers, dropout_rate, normalisation, skip_connection):
        super(GCNBiDirectional, self).__init__()

        # Set variables
        self.hidden_dim = hidden_dim
        self.num_convolutional_layers = num_convolutional_layers
        self.dropout_rate = dropout_rate
        self.skip_connection = skip_connection

        # Add convolutional layers
        self.convs_up = build_conv_model(num_convolutional_layers, hidden_dim, flow="source_to_target")
        self.convs_down = build_conv_model(num_convolutional_layers, hidden_dim, flow="target_to_source")

        # Add normalisation layers used in between graph convolutions
        if normalisation is None:
            self.lns = None
        else:
            self.normaliser = GCN_NORMALISATION[normalisation]
            self.lns = build_normalisation_layers(self.normaliser, hidden_dim, num_convolutional_layers - 1)

        # Add Linear layers
        self.linear = build_merge_linear_layers(num_convolutional_layers, hidden_dim)

    def forward(self, x, edge_index):

        # Iterate over each convolutional sequence
        for i in range(self.num_convolutional_layers):

            # Apply convolutions
            x_up = self.convs_up[i](x, edge_index)
            x_down = self.convs_down[i](x, edge_index)

            # Check if applying skip connection
            if self.skip_connection:
                x_up = x + x_up
                x_down = x + x_down

            # Concat convolutions
            x = torch.cat((x_up, x_down), dim=1)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

            # Merge through linear
            x = self.linear[i](x)
            emb = x
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

            # Normalise, if set
            if self.lns is not None and not i == self.num_convolutional_layers - 1:  # Apply normalisation
                x = self.lns[i](x)

        return emb, x


sub = GCNBiDirectional(
    hidden_dim=32, num_convolutional_layers=3, dropout_rate=0.25, normalisation="batch", skip_connection=False
)
sub

GCNBiDirectional(
  (convs_up): ModuleList(
    (0): GCNConv(32, 32)
    (1): GCNConv(32, 32)
    (2): GCNConv(32, 32)
  )
  (convs_down): ModuleList(
    (0): GCNConv(32, 32)
    (1): GCNConv(32, 32)
    (2): GCNConv(32, 32)
  )
  (lns): ModuleList(
    (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (linear): ModuleList(
    (0): Linear(64, 32, bias=True)
    (1): Linear(64, 32, bias=True)
    (2): Linear(64, 32, bias=True)
  )
)

In [16]:
class GNNStack(torch.nn.Module):
    def __init__(
        self,
        hidden_dim,
        num_convolutional_layers,
        no_dense_layers,
        direction,
        dropout_rate=0.0,
        task="premise",
        normalisation="layer",
        skip_connection=True,
    ):
        super(GNNStack, self).__init__()

        # Set variables
        self.task = task
        self.dropout_rate = dropout_rate
        self.hidden_dim = hidden_dim

        # Add embedding layer
        self.node_embedding = Embedding(len(config.NODE_TYPE), hidden_dim)

        # Add GCN layer
        if direction == "single":
            gcn_base = GCNDirectional
        elif direction == "separate":
            gcn_base = GCNBiDirectional
        else:
            raise ValueError(f"Unkown gcn direction {direction}")

        self.gcn = gcn_base(
            hidden_dim=self.hidden_dim,
            num_convolutional_layers=num_convolutional_layers,
            dropout_rate=self.dropout_rate,
            normalisation=normalisation,
            skip_connection=skip_connection,
        )

        # Post-message-passing
        self.post_mp = get_dense_output_network(
            no_dense_layers, hidden_dim, task=self.task, dropout_rate=self.dropout_rate
        )

    def forward(self, data):
        x, edge_index, premise_index = data.x, data.edge_index, data.premise_index

        x = self.node_embedding(x)

        emb, x = self.gcn(x, edge_index)

        if self.task == "premise":
            x = self.post_mp(x, premise_index)

            # elif self.task == 'graph':
            #    x = pyg_nn.global_mean_pool(x, batch)
        else:
            raise NotImplementedError()

        return emb, x

    #def loss(self, pred, label):
    #    return F.nll_loss(pred, label)


test_model = GNNStack(
    hidden_dim=32,
    num_convolutional_layers=3,
    no_dense_layers=1,
    direction="single",
    dropout_rate=0.25,
    task="premise",
)

In [17]:
print(test_model)

GNNStack(
  (node_embedding): Embedding(15, 32)
  (gcn): GCNDirectional(
    (convs): ModuleList(
      (0): GCNConv(32, 32)
      (1): GCNConv(32, 32)
      (2): GCNConv(32, 32)
    )
    (lns): ModuleList(
      (0): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
  )
  (post_mp): DensePremiseOutput(
    (lin): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
    )
    (out): Linear(in_features=32, out_features=1, bias=True)
  )
)


## Train and Test

In [18]:
def train(model, train_data, criterion, optimizer):

    model.train()

    for data in train_data:  # Iterate in batches over the training dataset.
        _, out = model(data)  # Perform a single forward pass. TODO change this

        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        

def test(model, test_data, writer, testing: bool):

    model.eval()

    correct = 0
    total_samples = 0
    total_loss = 0
    
    for data in test_data:  # Iterate in batches over the training/test dataset.
        _, out = model(data)
        pred = torch.sigmoid(out).round().long()

        correct += data.y.eq(pred).sum().item()

        loss = F.binary_cross_entropy_with_logits(out, data.y, reduction='sum')
        total_loss += loss
        total_samples += len(pred)
        
    acc_score = correct / total_samples # Derive ratio of correct predictions.
    total_loss /= total_samples
    total_loss = total_loss.item()

    
    if testing:
        writer.report_val_score(acc_score)
        writer.report_val_loss(total_loss)
    else:
        writer.report_train_score(acc_score)
        writer.report_train_loss(total_loss)
        
    return  acc_score

In [ ]:
def train_loop():
    
    train_data = get_data_loader(TRAIN_ID, BENCHMARK_TYPE, **dataset_params)
    val_data = get_data_loader(VAL_ID, BENCHMARK_TYPE, **dataset_params)

    model = GNNStack(
            hidden_dim=16,
            num_convolutional_layers=1,
            no_dense_layers=1,
            direction="single",
            dropout_rate=0.2,
            task="premise",
    )
    writer = Writer(model)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.BCEWithLogitsLoss(reduction="mean")
    #criterion = torch.nn.BCEWithLogitsLoss(reduction="none")


    print('\nTraining')
    for epoch in range(1, 5):


        train(model, train_data, criterion, optimizer)

        train_acc = test(model, train_data, writer, testing=False)
        test_acc = test(model, val_data, writer, testing=True)
        print(f"Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")



        writer.on_step()


    return writer
writer = train_loop()

Dataset: TorchMemoryDataset(22179)
Dataset: TorchMemoryDataset(2465)


2023-02-08 16:29:56.293760: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.



Training


In [ ]:
writer.get_scores()

In [ ]:
"""
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.node_embedding = Embedding(len(config.NODE_TYPE), hidden_channels)


        self.conv1 = GCNConv(hidden_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        
        self.linear = Linear(hidden_channels, 1)
        

    def forward(self, input_batch):
        
        x = input_batch.x
        edge_index = input_batch.edge_index
        premise_index = input_batch.premise_index    

        x = self.node_embedding(x)

        x = self.conv1(x, edge_index)
        x = x.relu()
        
        x = F.dropout(x, p=0.5, training=self.training) # TODO add dropout parameter?
        
        x = self.conv2(x, edge_index)
        x = x.relu()
        
        x = x[premise_index]
        x = self.linear(x)
        
        # Remove inner axis
        x = x.squeeze(-1)
        
    
        return x

    
model = GCN(hidden_channels=16)
print(model)
"""